In [2]:
# ============================================================
# INSTALL DEPENDENCIES
# ============================================================
!pip install -q langgraph reportlab

# ============================================================
# IMPORTS
# ============================================================
import random
import uuid
from datetime import datetime
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, END

from reportlab.platypus import SimpleDocTemplate, Paragraph, Table, Spacer
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.pagesizes import A4

# ============================================================
# INVENTORY (NO POPULARITY, STOCK AWARE)
# ============================================================
INVENTORY = {
    "shirts": [
        {"name": "Formal Shirt 1", "stock": 5, "price": 1999},
        {"name": "Formal Shirt 2", "stock": 0, "price": 1799},
        {"name": "Formal Shirt 3", "stock": 4, "price": 2199},
    ],
    "pants": [
        {"name": "Pant 1", "stock": 6, "price": 2499},
        {"name": "Pant 2", "stock": 3, "price": 2299},
        {"name": "Pant 3", "stock": 0, "price": 2699},
    ],
    "ethnic": [
        {"name": "Ethnic Wear 1", "stock": 2, "price": 3499},
        {"name": "Ethnic Wear 2", "stock": 5, "price": 2999},
    ],
    "athleisure": [
        {"name": "Athleisure Wear 1", "stock": 7, "price": 1599},
        {"name": "Athleisure Wear 2", "stock": 4, "price": 1799},
    ]
}

COMPLEMENTARY = {
    "shirts": ["pants"],
    "pants": ["shirts"],
    "ethnic": ["ethnic"],
    "athleisure": ["athleisure"]
}

# ============================================================
# HELPERS
# ============================================================
def norm(text):
    return text.strip().lower()

def available_products(category):
    return [i["name"] for i in INVENTORY[category] if i["stock"] > 0]

def smart_recommend(category, cart, top_k=2):
    cart_products = {item["product"] for item in cart}
    cart_prices = [item["price"] for item in cart]
    avg_price = sum(cart_prices) / len(cart_prices) if cart_prices else 2000

    candidates = []

    for cat in COMPLEMENTARY.get(category, []):
        for item in INVENTORY.get(cat, []):
            if item["stock"] <= 0:
                continue
            if item["name"] in cart_products:
                continue

            price_diff = abs(item["price"] - avg_price)
            score = 1 / (1 + price_diff)   # price affinity

            candidates.append({
                "product": item["name"],
                "price": item["price"],
                "score": score
            })

    candidates.sort(key=lambda x: x["score"], reverse=True)
    return [c["product"] for c in candidates[:top_k]]

# ============================================================
# STATE
# ============================================================
class FashionState(TypedDict):
    user_input: str
    step: str
    category: str
    product: str
    size: str
    cart: list
    cart_total: int
    discount: int
    final_price: int
    payment_attempts: int
    recommended_items: list
    response: str

# ============================================================
# PDF INVOICE
# ============================================================
def generate_invoice_pdf(state):
    invoice_id = str(uuid.uuid4())[:8]
    filename = f"invoice_{invoice_id}.pdf"

    doc = SimpleDocTemplate(filename, pagesize=A4)
    styles = getSampleStyleSheet()
    elements = []

    elements.append(Paragraph("<b>SMART FASHION STORE</b>", styles["Title"]))
    elements.append(Spacer(1, 10))
    elements.append(Paragraph(f"Invoice ID: {invoice_id}", styles["Normal"]))
    elements.append(Paragraph(f"Date: {datetime.now()}", styles["Normal"]))
    elements.append(Spacer(1, 10))

    table = [["Product", "Size", "Price"]]
    for item in state["cart"]:
        table.append([item["product"], item["size"], f"₹{item['price']}"])

    table.extend([
        ["", "Subtotal", f"₹{state['cart_total']}"],
        ["", "Discount", f"₹{state['discount']}"],
        ["", "Final Amount", f"₹{state['final_price']}"],
    ])

    elements.append(Table(table))
    doc.build(elements)
    return filename

# ============================================================
# AGENTS
# ============================================================
def ask_product(state):
    text = norm(state["user_input"])
    for cat in INVENTORY:
        if cat in text or cat[:-1] in text:
            state["category"] = cat
            state["response"] = (
                f"Available {cat}:\n" +
                ", ".join(available_products(cat)) +
                "\n\nWhich one would you like?"
            )
            state["step"] = "select_product"
            return state

    state["response"] = "What are you shopping for? (shirts / pants / ethnic / athleisure)"
    return state

def select_product(state):
    state["product"] = state["user_input"]
    state["price"] = next(
        (i["price"] for i in INVENTORY[state["category"]] if i["name"] == state["product"]),
        random.choice([1499, 1999, 2499])
    )
    state["response"] = "Select size: S / M / L / XL"
    state["step"] = "select_size"
    return state

def select_size(state):
    state["size"] = state["user_input"].upper()
    state["response"] = "Add to cart? (yes / no)"
    state["step"] = "cart_decision"
    return state

def cart_decision(state):
    if norm(state["user_input"]) == "yes":
        state["cart"].append({
            "product": state["product"],
            "size": state["size"],
            "price": state["price"]
        })
        state["cart_total"] += state["price"]

        recs = smart_recommend(state["category"], state["cart"])
        if recs:
            state["recommended_items"] = recs
            state["response"] = (
                "Item added to cart 🛒\n\n"
                "Recommended for you:\n" +
                ", ".join(recs) +
                "\n\nAdd a recommended item? (yes / no)"
            )
            state["step"] = "recommendation"
            return state

        state["response"] = "Item added to cart 🛒\nShop more? (yes / no)"
        state["step"] = "shop_more"
        return state

    state["response"] = "Okay 🙂 What else would you like to shop for?"
    state["step"] = "ask_product"
    return state

def recommendation(state):
    if norm(state["user_input"]) == "yes" and state["recommended_items"]:
        item = state["recommended_items"][0]
        price = next(
            (i["price"] for cat in INVENTORY for i in INVENTORY[cat] if i["name"] == item),
            999
        )
        state["cart"].append({"product": item, "size": "M", "price": price})
        state["cart_total"] += price
        state["response"] = f"{item} added 👍\nShop more? (yes / no)"
    else:
        state["response"] = "No problem 🙂 Shop more? (yes / no)"

    state["step"] = "shop_more"
    return state

def shop_more(state):
    if norm(state["user_input"]) == "yes":
        state["step"] = "ask_product"
        state["response"] = "What would you like next?"
        return state

    summary = "\n".join([f"{i['product']} – ₹{i['price']}" for i in state["cart"]])
    state["response"] = (
        "🛒 CART SUMMARY:\n" + summary +
        f"\n\nSubtotal: ₹{state['cart_total']}\n\n"
        "Offers:\n1) HDFC ₹300\n2) ICICI ₹250\n3) SBI ₹200\nChoose offer:"
    )
    state["step"] = "apply_offer"
    return state

def apply_offer(state):
    offers = {"1": 300, "2": 250, "3": 200}
    state["discount"] = offers.get(state["user_input"], 0)
    state["final_price"] = state["cart_total"] - state["discount"]
    state["response"] = f"Final amount: ₹{state['final_price']}\nBuy online or store?"
    state["step"] = "payment"
    return state

def payment(state):
    choice = state["user_input"].lower()

    if choice == "store":
        state["response"] = "Please complete payment at the nearest store."
        state["step"] = "support"
        return state

    # ONLINE PAYMENT
    state["payment_attempts"] += 1
    success = random.random() < 0.7

    if success:
        state["response"] = "✅ Payment successful! Delivery in 3–5 days."
        state["step"] = "support"
        return state

    if state["payment_attempts"] == 1:
        state["response"] = "❌ Payment failed. Retry? (yes / no)"
        state["step"] = "payment"
        return state

    state["response"] = "❌ Payment failed twice. Please pay at store."
    state["step"] = "support"
    return state


def support(state):
    invoice_text, filename = generate_invoice(state)

    state["response"] = (
        "✅ Order completed successfully!\n\n"
        "🧾 INVOICE:\n"
        f"{invoice_text}\n"
        f"📁 Invoice saved as: {filename}\n\n"
        "⭐ Please rate your experience (1–5):"
    )

    state["step"] = "csat"
    return state

def generate_invoice(state):
    invoice_id = str(uuid.uuid4())[:8]
    date = datetime.now().strftime("%d-%m-%Y %H:%M")

    items = "\n".join(
        [f"{i+1}. {item['product']} (Size {item['size']}) - ₹{item['price']}"
         for i, item in enumerate(state["cart"])]
    )

    invoice_text = f"""
==============================
        SMART FASHION STORE
==============================
Invoice ID : {invoice_id}
Date       : {date}

Items Purchased:
{items}

------------------------------
Subtotal      : ₹{state['cart_total']}
Discount      : ₹{state['discount']}
Final Amount  : ₹{state['final_price']}
------------------------------

Thank you for shopping with us!
==============================
"""

    # Save invoice to file
    filename = f"invoice_{invoice_id}.txt"
    with open(filename, "w") as f:
        f.write(invoice_text)

    return invoice_text, filename

# ============================================================
# ADDED CSAT FUNCTION
# ============================================================
def csat(state):
    # For now, just acknowledge the rating and end the chat.
    # More complex logic (e.g., storing rating) could be added here.
    rating = state["user_input"]
    state["response"] = f"Thank you for your {rating}-star rating! Have a great day!"
    state["step"] = "end" # Transition to 'end' to stop the chat loop
    return state

# ============================================================
# LANGGRAPH (STRUCTURAL)
# ============================================================
graph = StateGraph(FashionState)
for fn in [
    ask_product, select_product, select_size,
    cart_decision, recommendation, shop_more,
    apply_offer, payment, support, csat # Added csat here
]:
    graph.add_node(fn.__name__, fn)

graph.set_entry_point("ask_product")
for n in [
    "ask_product","select_product","select_size","cart_decision",
    "recommendation","shop_more","apply_offer","payment","support", "csat" # Added csat here
]:
    # The actual graph edges are not fully utilized by the manual chat loop,
    # but adding it here for consistency if graph traversal were implemented.
    if n != "csat": # csat should transition to END
        graph.add_edge(n, END)
    else:
        graph.add_edge(n, END) # The csat function already sets state['step'] = 'end', so this would be redundant if using graph.invoke

graph.compile()

# ============================================================
# CHAT LOOP
# ============================================================
NODE_MAP = {
    "ask_product": ask_product,
    "select_product": select_product,
    "select_size": select_size,
    "cart_decision": cart_decision,
    "recommendation": recommendation,
    "shop_more": shop_more,
    "apply_offer": apply_offer,
    "payment": payment,
    "support": support,
    "csat": csat # Added the new csat function to the NODE_MAP
}

def fashion_chatbot():
    state = {
        "user_input": "",
        "step": "ask_product",
        "category": "",
        "product": "",
        "size": "",
        "cart": [],
        "cart_total": 0,
        "discount": 0,
        "final_price": 0,
        "payment_attempts": 0,
        "recommended_items": [],
        "response": ""
    }

    print("👗 SMART FASHION STORE AI (type 'exit')")
    while True:
        user = input("\nYou: ")
        if norm(user) == "exit":
            break

        state["user_input"] = user
        state = NODE_MAP[state["step"]](state)
        print("\n🤖 Assistant:\n", state["response"])

        if state["step"] == "end":
            break

fashion_chatbot()


👗 SMART FASHION STORE AI (type 'exit')

You: naku

🤖 Assistant:
 What are you shopping for? (shirts / pants / ethnic / athleisure)

You: pants

🤖 Assistant:
 Available pants:
Pant 1, Pant 2

Which one would you like?

You: pant 1

🤖 Assistant:
 Select size: S / M / L / XL

You: M

🤖 Assistant:
 Add to cart? (yes / no)

You: yes

🤖 Assistant:
 Item added to cart 🛒

Recommended for you:
Formal Shirt 1, Formal Shirt 3

Add a recommended item? (yes / no)

You: no

🤖 Assistant:
 No problem 🙂 Shop more? (yes / no)

You: no

🤖 Assistant:
 🛒 CART SUMMARY:
pant 1 – ₹1999

Subtotal: ₹1999

Offers:
1) HDFC ₹300
2) ICICI ₹250
3) SBI ₹200
Choose offer:

You: SBI

🤖 Assistant:
 Final amount: ₹1999
Buy online or store?

You: store

🤖 Assistant:
 Please complete payment at the nearest store.

You: ok

🤖 Assistant:
 ✅ Order completed successfully!

🧾 INVOICE:

        SMART FASHION STORE
Invoice ID : 655af890
Date       : 17-12-2025 13:37

Items Purchased:
1. pant 1 (Size M) - ₹1999

------------------